<div class="alert alert-block alert-info">
<b>Define Layer</b> 3 types of layer class are defined : (1) input layer, (2) dense layer, and (3) activation layer</div>

In [1]:
class Input_layer:
    def __init__(self, input_depth, input_width=1):
        self.TYPE = 'INPUT'
        self.SHAPE = (input_depth, input_width)
        a = list()
        for i in range(self.SHAPE[0]):
            tmp = []
            for j in range(self.SHAPE[1]):
                tmp.append(f'I_{i},{j}')
            a.append(tmp)
        self.VALUE = a

class Dense_layer:
    def __init__(self,size_of_node):
        self.TYPE = 'DENSE'
        self.SIZE = size_of_node
        self.INPUT_SIZE = 0
        self.WEIGHT = list()
        self.BIAS = list()

    def __call__(self,input_size,rank=None):
        self.INPUT_SIZE = input_size
        
        for row in range(self.SIZE):
            weight_tmp = list()
            for col in range(self.INPUT_SIZE):
                weight_tmp.append(f'W{rank}_{row},{col}')
            self.WEIGHT.append(weight_tmp)

        bias_tmp = list()
        for row in range(self.SIZE):
            bias_tmp.append(f'B{rank}_{row}')
        self.BIAS = bias_tmp
        return self

class Activation_layer:
    def __init__(self, function):
        self.TYPE = 'ACTIVATION'
        self.function = function

<div class="alert alert-block alert-info">
<b>Define Model </b> Design a model that takes layers as parameters. Layers will be stacked in as list</div>

In [2]:
class Sequential_model:
    def __init__(self, *list_of_layers):
        self.dense_counter = 0
        self.activation_counter = 0
        self.input_size = None
        self.model_layer = {}

        for layer in list_of_layers:
            if layer.TYPE == 'INPUT':
                self.model_layer['input_layer'] = layer.VALUE
                self.input_size = layer.SHAPE[1]

            elif layer.TYPE == 'DENSE':
                self.dense_counter += 1
                layer(self.input_size,self.dense_counter)
                self.model_layer[f'dense_layer_{self.dense_counter}'] = (layer.WEIGHT, layer.BIAS)
                self.input_size = layer.SIZE

            elif layer.TYPE == 'ACTIVATION':
                self.activation_counter += 1
                self.model_layer[f'activation_layer_{self.activation_counter}'] = layer.function
    def summary(self):
        for layer in self.model_layer.keys():
            print(f'{layer} : {self.model_layer[layer]}')

    def forward(self):
        self.input_layer = []
        for key in self.model_layer.keys():
            if 'input' in key:
                self.input_layer = self.model_layer[key]
                # self.input_depth = len(self.model_layer[key])
                # self.input_width = len(self.model_layer[key][0])
                
            elif 'dense' in key:
                signal_out = list()
                weight_layer = self.model_layer[key][0]
                bias_layer = self.model_layer[key][1]
                depth = len(weight_layer)
                
                for row in range(depth):
                    mul_str = ''
                    for input_row in range(len(self.input_layer)):
                        for input_col in range(len(self.input_layer[0])):
                            mul_str += weight_layer[row][input_row] +'*'+ self.input_layer[input_row][input_col]+ '+' + bias_layer[row] + '+'
                    mul_str = mul_str[:-1]
                    signal_out.append([mul_str])
                self.input_layer = signal_out
            
            elif 'activation' in key:
                signal_out = self.input_layer
                for row in range(len(self.input_layer)):
                    for col in range(len(self.input_layer[0])):
                        signal_out[row][col] = f'{self.model_layer[key]}({self.input_layer[row][col]})'
                self.input_layer = signal_out
        
        return signal_out

<div class="alert alert-block alert-info">
<b>Config Model </b> Define input and Output size and stack layers </div>

In [3]:
# VARIABLE SETTING AND LIMITER
INPUT_SIZE = 4
OUTPUT_SIZE = 10

# CONFIG MODEL
model = Sequential_model(
    Input_layer(1,INPUT_SIZE),
    Dense_layer(2),
    Activation_layer('Relu'),
    Dense_layer(4),
    Activation_layer('Relu'),
    Dense_layer(OUTPUT_SIZE),
    Activation_layer('Softmax')
)

<div class="alert alert-block alert-info">
<b>Limiter </b> Just in Case, if the parameter is too large, numerical gradient will not work. To prevent OOM error, limit the size of the nodes for each layers</div>

In [4]:
# MODEL LIMITER
for key in model.model_layer.keys():
    if 'dense' in key:
        if len(model.model_layer[key][0])*len(model.model_layer[key][0][0]) < 100:
            pass
        else:
            raise MemoryError(f'too large to calculate numerical gradient in {key} ')
print('REQUIREMENT : PASSED')

REQUIREMENT : PASSED


<div class="alert alert-block alert-info">
<b>Model Summary </b> model layers are saved as dictionary</div>

In [5]:
# MODEL SUMMARY
model.summary()

input_layer : [['I_0,0', 'I_0,1', 'I_0,2', 'I_0,3']]
dense_layer_1 : ([['W1_0,0', 'W1_0,1', 'W1_0,2', 'W1_0,3'], ['W1_1,0', 'W1_1,1', 'W1_1,2', 'W1_1,3']], ['B1_0', 'B1_1'])
activation_layer_1 : Relu
dense_layer_2 : ([['W2_0,0', 'W2_0,1'], ['W2_1,0', 'W2_1,1'], ['W2_2,0', 'W2_2,1'], ['W2_3,0', 'W2_3,1']], ['B2_0', 'B2_1', 'B2_2', 'B2_3'])
activation_layer_2 : Relu
dense_layer_3 : ([['W3_0,0', 'W3_0,1', 'W3_0,2', 'W3_0,3'], ['W3_1,0', 'W3_1,1', 'W3_1,2', 'W3_1,3'], ['W3_2,0', 'W3_2,1', 'W3_2,2', 'W3_2,3'], ['W3_3,0', 'W3_3,1', 'W3_3,2', 'W3_3,3'], ['W3_4,0', 'W3_4,1', 'W3_4,2', 'W3_4,3'], ['W3_5,0', 'W3_5,1', 'W3_5,2', 'W3_5,3'], ['W3_6,0', 'W3_6,1', 'W3_6,2', 'W3_6,3'], ['W3_7,0', 'W3_7,1', 'W3_7,2', 'W3_7,3'], ['W3_8,0', 'W3_8,1', 'W3_8,2', 'W3_8,3'], ['W3_9,0', 'W3_9,1', 'W3_9,2', 'W3_9,3']], ['B3_0', 'B3_1', 'B3_2', 'B3_3', 'B3_4', 'B3_5', 'B3_6', 'B3_7', 'B3_8', 'B3_9'])
activation_layer_3 : Softmax


<div class="alert alert-block alert-info">
<b>Error Calculation </b> MSE is used as error for convinience</div>

In [6]:
# CALCULATE MSE ERROR
output = model.forward()

truth = [f'T{i}' for i in range(OUTPUT_SIZE)]

tmp = ''
for i in range(OUTPUT_SIZE):
    tmp += f'({truth[i]}-{output[i][0]})^2 +'

mse_error = f'1/2 * [{tmp[:-1]}]'

<div class="alert alert-block alert-info">
<b>Numerical Gradient Calculation</b> calculates gradients for the error on each node in each layers</div>

In [7]:
# NUMERICAL GRADIENT
import copy
numerical_gradient = copy.deepcopy(model.model_layer)

for key in model.model_layer.keys():
    if 'input' in key:
        pass

    elif 'dense' in key:
        weight = model.model_layer[key][0]
        for row in range(len(weight)):
            for col in range(len(weight[0])):
                word = weight[row][col]
                tmp1 = output
                tmp2 = output
                numerical_gradient[key][0][row][col] = f"{tmp.replace(word, f'({word} + H)')} - {tmp.replace(word, f'({word} - H)')} / 2H"

        bias = model.model_layer[key][1]
        for row in range(len(bias)):
            word = bias[row]
            tmp1 = output
            tmp2 = output
            numerical_gradient[key][1][row] = f"{tmp.replace(word, f'({word} + H)')} - {tmp.replace(word, f'{word} - H')} / 2H"
            
    elif 'activation' in key:
        word = model.model_layer[key]
        numerical_gradient[key] = f'derivative of {word}'


<div class="alert alert-block alert-info">
<b>Weight update</b> LR : Learning Rate</div>

In [8]:
# UPDATE
updated_model_layer = copy.deepcopy(model.model_layer)
for key in model.model_layer.keys():
    if 'input' in key:
        pass

    elif 'dense' in key:
        weight = model.model_layer[key][0]
        for row in range(len(weight)):
            for col in range(len(weight[0])):
                word = weight[row][col]
                updated_model_layer[key][0][row][col] = f'{word} -LR*{numerical_gradient[key][0][row][col]}'
        
        bias = model.model_layer[key][1]
        for row in range(len(bias)):
            word = bias[row]
            updated_model_layer[key][1][row] = f'{word} - LR*{numerical_gradient[key][1][row]}'

    elif 'activation' in key:
        pass


<div class="alert alert-block alert-info">
<b>View updated weight</b> since the equation is very long, it is recommanded to specify the exact layer you are looking for</div>

In [9]:

eq = updated_model_layer['dense_layer_1'][0][0][0]

<div class="alert alert-block alert-info">
<b>save and load equation</b> try python prompt does not print all equation</div>

In [10]:

# SAVE EQUATION IN PICKLE
import pickle
filename = './equation_for_updating_single_node.pkl'
with open(filename, 'wb') as f:
    pickle.dump(eq,f)


In [ ]:

# LOAD EQUATION FROM PICKLE
with open(filename,'rb') as f:
    x = pickle.load(f)
    print(x)